Dicionário de mensagens

In [19]:
mensagem = {"menu": """
            A. Mostrar dataset
            B. Quantidade de Homens e Mulheres
            C. Quantidade de Sobreviventes e Mortos
            D. Quantidade de mulheres que não Sobreviveram
            E. Proporção de sobreviventes entre homens e mulheres
            F. Idade com o maior número de mortos
            G. Média de idade dos homens sobreviventes
            H. Proporção de sobreviventes por sexo (passageiros prioritários)
            I. Quantidade de passageiro por classe
            J. Porcentagem de sobrevivente por classe
            K. Relação de sobreviventes e mortos agrupados por classe e sexo
            L. Relação de sobreviventes e mortos de homens com idade entre 24 e 30 anos
            M. Probabilidade de sobrevivência
            S. Sair
            """,
            "menuDataset": """
            A. Informações das Colunas
            B. Dados principais
            """
        }

menu = {
      "A" : "Mostrar dataset",
      "B" : "Quantidade de Homens e Mulheres",
      "C" : "Quantidade que Sobreviveram e Não sobreviveram",
      "D" : "Quantidade de mulheres que não Sobreviveram",
      "E" : "Proporção de sobreviventes entre homens e mulheres",
      "F" : "Idade com o maior número de mortos",
      "G" : "Média de idade dos homens sobreviventes",
      "H" : "Proporção de sobreviventes por sexo (passageiros prioritários)",
      "I" : "Quantidade de passageiro por classe",
      "J" : "Porcentagem de sobrevivente por classe",
      "K" : "Relação de sobreviventes e mortos agrupados por classe e sexo",
      "L" : "Relação de sobreviventes e mortos de homens com idade entre 24 e 30 anos",
      "M" : "Probabilidade de sobrevivência ",
      "S" : "Sair"
      }

Gerenciador de arquivos para adequar o dataset

In [36]:
import numpy as np
import pandas as pd

class GerenciarArquivos():
  #no próprio construtor ele já processa e salva os arquivos para garantir que sempre estará atualizado
  def __init__(self):
    self.processarArquivos()

  def processarArquivos(self):
    dataset = pd.read_csv('/content/titanic_data.csv')

    valores_preenchimento = {
    'Age': np.random.randint(1, 100),
    'Cabin': 'Unknown',
    'Embarked': 'S',
    }

    dataset_preenchido = dataset.fillna(value=valores_preenchimento)

    dataset_adequado = dataset_preenchido.drop(['SibSp', 'Parch', 'Ticket'], axis=1)

    novos_nomes = {
    'PassengerId': 'IdPassageiro',
    'Survived': 'Sobreviveu',
    'Pclass': 'Classe',
    'Name': 'Nome',
    'Sex': 'Sexo',
    'Age': 'Idade',
    'Fare': 'Tarifa',
    'Cabin': 'Cabine',
    'Embarked': 'Embarque',
    }
    dataset_adequado = dataset_adequado.rename(columns=novos_nomes)

    coluna_sobreviveu = dataset_adequado['Sobreviveu']
    nova_coluna_sobreviveu = coluna_sobreviveu.apply(self.change_sobreviveu)
    dataset_adequado['Sobreviveu'] = nova_coluna_sobreviveu

    coluna_sexo= dataset_adequado['Sexo']
    nova_coluna_sexo = coluna_sexo.apply(self.change_sexo)
    dataset_adequado['Sexo'] = nova_coluna_sexo
    dataset_adequado.to_csv('/content/novo_titanic_data.csv', index=False)

  def mostrarInfo(self):
    dataset = pd.read_csv('/content/titanic_data.csv')
    dataset.info()

  def mostrarDados(self):
    dataset = pd.read_csv('/content/titanic_data.csv')
    ds = pd.merge(dataset.head(10), dataset.tail(8), how = 'outer')
    df = pd.DataFrame(ds)
    return df


  def change_sobreviveu(self, x):
    if x == 0:
        x = 'Não'
    else:
        x = 'Sim'
    return x

  def change_sexo(self, x):
    if x == 'female':
        x = 'Mulher'
    else:
        x = 'Homem'

    return x

Estatisticas

In [12]:
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt

class Estatisticas():
  dataset_adequado = {}
  def __init__(self):
    GerenciarArquivos()
    self.dataset_adequado = pd.read_csv('/content/novo_titanic_data.csv')

  def qtdHomensMulheres(self):
    return self.dataset_adequado['Sexo'].value_counts()

  def qtdSobreviveram(self):
    return self.dataset_adequado['Sobreviveu'].value_counts()

  def qtdMulheres(self):
    qtd = self.dataset_adequado[
    (self.dataset_adequado['Sexo'] == 'Mulher') & (self.dataset_adequado['Sobreviveu'] == 'Não')
    ].shape[0]
    return {"Mulher": qtd}

  def proporcaoSobrevivente(self):
    quantidade_M_S = self.dataset_adequado[
      (self.dataset_adequado['Sexo'] == 'Mulher') & (self.dataset_adequado['Sobreviveu'] == 'Sim')
      ].shape[0]

    quantidade_H_S = self.dataset_adequado[
    (self.dataset_adequado['Sexo'] == 'Homem') & (self.dataset_adequado['Sobreviveu'] == 'Sim')
    ].shape[0]

    total_passageiros = len(self.dataset_adequado[
      self.dataset_adequado['Sobreviveu'] == 'Sim'
    ])

    porcentagem_M_S = quantidade_M_S/total_passageiros
    porcentagem_H_S = quantidade_H_S/total_passageiros

    return { "Homem": f"{porcentagem_H_S: .2%}",
            "Mulher": f"{porcentagem_M_S: .2%}"
            }

  def idadeMaxMortos(self):
    numero_nao_sobreviventes_idade = self.dataset_adequado[
    (self.dataset_adequado['Sobreviveu'] == 'Não')
    ]['Idade'].value_counts()

    return {"Idade com maior número de mortos": int(numero_nao_sobreviventes_idade.idxmax()),
            "Quantidade": numero_nao_sobreviventes_idade.max()
            }

  def idadeMediaHomens(self):
    idade_media_homens = self.dataset_adequado[
    (self.dataset_adequado['Sexo'] == 'Homem') & (self.dataset_adequado['Sobreviveu'] == 'Sim')
    ]

    return {"Media": idade_media_homens['Idade'].mean()}

  def proporcaoSobreviventePrioritario(self):
    quan_pass_prioritarios_total = self.dataset_adequado[
    ((self.dataset_adequado['Idade'] <= 15) | (self.dataset_adequado['Sexo'] == 'Mulher')) & (self.dataset_adequado['Sobreviveu'] == 'Sim')
    ]

    proporcao_sobreviventes = (quan_pass_prioritarios_total.groupby('Sexo')['Sobreviveu'].count())
    proporcao_sobreviventes_homem = proporcao_sobreviventes[0]/len(quan_pass_prioritarios_total)
    proporcao_sobreviventes_mulher = proporcao_sobreviventes[1]/len(quan_pass_prioritarios_total)

    return {"Homem": f"{proporcao_sobreviventes_homem: .2%}",
            "Mulher": f"{proporcao_sobreviventes_mulher: .2%}"
            }

  def qtdPassageiroClasse(self):
    quantidades = self.dataset_adequado['Classe'].value_counts()
    resposta = {}
    for i in range(1, 4):
      resposta.update({f"{i}º Classe": quantidades[i]})

    return resposta

  def porcentagemPassageiroClasse(self):
    quantidade_passageiros_classe = self.dataset_adequado['Classe'].value_counts()
    quantidade_sobreviventes_classe = self.dataset_adequado[
    (self.dataset_adequado['Sobreviveu'] == 'Sim')
    ]['Classe'].value_counts()

    percentual_classe = []
    for i in range(1,4):
        percentual_classe.append((quantidade_sobreviventes_classe[i])/(quantidade_passageiros_classe[i]))

    dict_porc_classe = {
        'Classe': ['Classe 1', 'Classe 2', 'Classe 3'],
        'Percentual': np.array(percentual_classe)*100,
    }

    porcentagem_classe_df = pd.DataFrame.from_dict(dict_porc_classe)

    ax = porcentagem_classe_df.plot(kind='bar', xlabel='Classe', ylabel='Percentual de sobreviventes', title='Percentual de Sobreviventes por Classe')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(100))
    ax.set_xticklabels(['Classe 1', 'Classe 2', 'Classe 3'])
    for container in ax.containers:
        ax.bar_label(container, fmt='%1.2f%%')
    plt.show()

    return porcentagem_classe_df

  def relacaoClasseSexo(self):
    df_sobreviventes_classe_sexo = self.dataset_adequado.groupby(by=['Sexo', 'Classe'])['Sobreviveu'].apply(lambda x:( x=='Sim').sum())
    df_n_sobreviventes_classe_sexo = self.dataset_adequado.groupby(by=['Sexo', 'Classe'])['Sobreviveu'].apply(lambda x: (x=='Não').sum())

    df_agrupado = pd.DataFrame({'Sobreviventes': df_sobreviventes_classe_sexo, 'Não Sobreviventes': df_n_sobreviventes_classe_sexo})
    ax = df_agrupado.plot(kind='bar', ylabel='Quantidade de Passageiros').set_xticklabels(['Homem e Classe 1', 'Homem e Classe 2', 'Homem e Classe 3','Mulher e Classe 1', 'Mulher e Classe 2', 'Mulher e Classe 3' ])
    plt.show()

    return df_agrupado

  def relacaoHomensIdade(self):
    quantidade_homens_24_30_classe_3 = self.dataset_adequado[
    (self.dataset_adequado['Idade']>24) &
    (self.dataset_adequado['Idade']<30) &
    ((self.dataset_adequado['Classe']==3)) &
    ((self.dataset_adequado['Sobreviveu']=='Sim')) &
    (self.dataset_adequado['Sexo']=='Homem')
    ].shape[0]

    quantidade_homens_24_30_classe_2 = self.dataset_adequado[
    (self.dataset_adequado['Idade']>24) &
    (self.dataset_adequado['Idade']<30) &
    ((self.dataset_adequado['Classe']==2)) &
    ((self.dataset_adequado['Sobreviveu']=='Não')) &
    (self.dataset_adequado['Sexo']=='Homem')
    ].shape[0]

    return {"Sobreviveram na 3º Classe": quantidade_homens_24_30_classe_3, "Não sobreviveram na 2º Classe": quantidade_homens_24_30_classe_2}

  def P(self, genero, classe):
    df_classe_sexo = self.dataset_adequado.groupby(['Sexo', 'Classe']).size()
    df_classe_sexo_sobreviventes = self.dataset_adequado[
        (self.dataset_adequado['Sobreviveu'] == 'Sim')
    ].groupby(['Sexo', 'Classe']).size()

    probabilidade_cond = df_classe_sexo_sobreviventes[genero][classe]/df_classe_sexo[genero][classe]
    ax = (df_classe_sexo_sobreviventes/df_classe_sexo).plot(kind='bar', ylabel='Probabilidade Condicional de Sobrevivência')
    ax.set_title('Probabilidade Condicional de Sobrevivência por Sexo e Classe')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.set_xticklabels(['Homem e Classe 1', 'Homem e Classe 2', 'Homem e Classe 3','Mulher e Classe 1', 'Mulher e Classe 2', 'Mulher e Classe 3'])
    plt.show()

    return '{:,.2%}'.format(probabilidade_cond)

  def mostrarDados(self, dados):
    for i in dict(dados):
      print(str(i) + ": "+str(dados[i]))



Menu

In [37]:
from google.colab import output
#menu

opcao = ""
while opcao != "S":
  opcao = input(mensagem["menu"]).upper()

  if opcao != "S":
    output.clear()
    if opcao in menu:
      print(menu[opcao])
      est = Estatisticas()

      if opcao == "A":
        opcaoDataset = input(mensagem["menuDataset"]).upper()

        if opcaoDataset == "A":
          output.clear()
          GerenciarArquivos().mostrarInfo()

        elif opcaoDataset == "B":
          output.clear()
          display(GerenciarArquivos().mostrarDados())
        else:
          print("Opção Inválida!")

      elif opcao == "B":
        est.mostrarDados(est.qtdHomensMulheres())

      elif opcao == "C":
        est.mostrarDados(est.qtdSobreviveram())

      elif opcao == "D":
        est.mostrarDados(est.qtdMulheres())

      elif opcao == "E":
        est.mostrarDados(est.proporcaoSobrevivente())

      elif opcao == "F":
        est.mostrarDados(est.idadeMaxMortos())

      elif opcao == "G":
        est.mostrarDados(est.idadeMediaHomens())

      elif opcao == "H":
        est.mostrarDados(est.proporcaoSobreviventePrioritario())

      elif opcao == "I":
        est.mostrarDados(est.qtdPassageiroClasse())

      elif opcao == "J":
        print(est.porcentagemPassageiroClasse())

      elif opcao == "K":
        print(est.relacaoClasseSexo())

      elif opcao == "L":
        est.mostrarDados(est.relacaoHomensIdade())

      elif opcao == "M":
        genero = input("Digite o seu genero(M/F): ").upper()

        if genero == "M":
           sexo = "Homem"
        elif genero == "F":
          sexo = "Mulher"
        else:
          print("Gênero Inválido!")

        classe = input("Digite a classe que vc embarcaria ")
        if classe == "1" or classe == "2" or classe == "3":
          print(f"A probabilidade de sobrevivência é {est.P(sexo, int(classe))}")
        else:
          print("Classe Inválida!")
    else:
      print("Opção Inválida!")









,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C



            A. Mostrar dataset
            B. Quantidade de Homens e Mulheres
            C. Quantidade de Sobreviventes e Mortos
            D. Quantidade de mulheres que não Sobreviveram
            E. Proporção de sobreviventes entre homens e mulheres
            F. Idade com o maior número de mortos
            G. Média de idade dos homens sobreviventes
            H. Proporção de sobreviventes por sexo (passageiros prioritários)
            I. Quantidade de passageiro por classe
            J. Porcentagem de sobrevivente por classe
            K. Relação de sobreviventes e mortos agrupados por classe e sexo
            L. Relação de sobreviventes e mortos de homens com idade entre 24 e 30 anos
            M. Probabilidade de sobrevivência
            S. Sair
            s
